In [1]:
import pandas as pd
import numpy as np
from bs4 import  BeautifulSoup
import requests

Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [2]:
#get data
wiki_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page = requests.get(wiki_link).text

#Parse HTML with ".prettyfy"
soup = BeautifulSoup(raw_wikipedia_page,'xml')
#Printing makes page too long
#print(soup.prettify())
# extracting the raw table inside that webpage
table = soup.find('table')

#Dictionary the columns
Postcode      = []
Borough       = []
Neighbourhood = []

#print table makes the page too long(table)

In [ ]:
Use the Notebook to build the code to scrape the Toronto Postal Code Wikipedia page

In [3]:
# Extracting a clean form of the table
for tr_cell in table.find_all('tr'):
    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1
    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')
            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')
            
        counter +=1
        #Clean up Unassigned 
    if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'): 
        continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue
        
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

In [4]:
unique_p = set(Postcode)
print('Total Unique Postal Codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []


for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

Total Unique Postal Codes: 77


In [5]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighborhood':Neighbourhood_u}
df_toronto = pd.DataFrame.from_dict(toronto_dict)
#Write to CSV 
df_toronto.to_csv('toronto_scraper.csv')
df_toronto.head()

,Postcode,Borough,Neighborhood
0,M9A,Etobicoke,Islington Avenue
1,M4H,East York,Thorncliffe Park
2,M1B,Scarborough,"Rouge, Malvern"
3,M9L,North York,Humber Summit
4,M4Y,Downtown Toronto,Church and Wellesley


Create the Dataframe as requested by the assignment

Follow New York City Geo Lab

In [32]:
#Shape of the dataframe
df_toronto.shape

(77, 3)

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON Files
from geopy.geocoders import Nominatim # covert an address into lat and long

from pandas.io.json import json_normalize # transfrom JSON file into a pandas dataframe

#import matplotlib.cm as cm
import matplotlib.colors as colors

#K Means Clustering
from sklearn.cluster import KMeans
import folium 
print ('Libraries imported!')

Libraries imported!


Geopy Library

In [7]:
!pip install geocoder
import geocoder # import geocoder

    100% |████████████████████████████████| 102kB 17.1MB/s 


In [8]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


Create map of Toronto with neighborhoods superimposed on top.

In [9]:
toronto_geo = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u,
              'Latitude': latitude, 'Longitude':longitude}
toronto = pd.DataFrame.from_dict(toronto_geo)
toronto.to_csv('toronto.csv')
toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.653963,-79.387207
1,M4H,East York,Thorncliffe Park,43.653963,-79.387207
2,M1B,Scarborough,"Rouge, Malvern",43.653963,-79.387207
3,M9L,North York,Humber Summit,43.653963,-79.387207
4,M4Y,Downtown Toronto,Church and Wellesley,43.653963,-79.387207
5,M9N,York,Weston,43.653963,-79.387207
6,M3J,North York,"Northwood Park, York University",43.653963,-79.387207
7,M2H,North York,Hillcrest Village,43.653963,-79.387207
8,M2J,North York,Henry Farm,43.653963,-79.387207
9,M5S,Downtown Toronto,University of Toronto,43.653963,-79.387207


In [10]:
#Loop Stuck. Tried Several Attempts over Several Days. Will use CSV file provided
#latitude = []
#longitude = []
#for elem in Postcode_u:
# initialize your variable to None
  #  lat_lng_coords = None
# loop until you get the coordinates
  #while(lat_lng_coords is None):
     #   g = geocoder.google('{}, Toronto, Ontario'.format(elem))
     #   lat_lng_coords = g.latlng
    
   # latitude = lat_lng_coords[0]
  #  longitude = lat_lng_coords[1]
#    print(elem, 'is RECEIVED.')
    # print(lat_lng_coords[0])
    # print(lat_lng_coords[1])

In [ ]:
We need to update get the unique Long and Lat for each neighborhood. The Geopy loop provided is not working. We may need to try alternative methods.

In [11]:
#Create new dataframe based on the CSV provided by the instructor
df = (pd.read_csv("Geospatial_Coordinates.csv"))

Lets Merge our scraped dataframe from wikipedia with the CSV provided by the instructor.
This way we will have fixed the Longitude and Latitude

In [12]:
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#Replace Long and Lat with data from Geospatial_Data Spreadsheet

In [14]:
#Rename Postal Code to "Postcode"
df = df.rename(columns={'Postal Code': 'Postcode'})

In [15]:
#Drop Old Lat and Long from Toronto dataframe
toronto.drop(['Longitude'], axis = 1,inplace=True)
toronto.drop(['Latitude'], axis = 1,inplace=True)
toronto

,Postcode,Borough,Neighbourhood
0,M9A,Etobicoke,Islington Avenue
1,M4H,East York,Thorncliffe Park
2,M1B,Scarborough,"Rouge, Malvern"
3,M9L,North York,Humber Summit
4,M4Y,Downtown Toronto,Church and Wellesley
5,M9N,York,Weston
6,M3J,North York,"Northwood Park, York University"
7,M2H,North York,Hillcrest Village
8,M2J,North York,Henry Farm
9,M5S,Downtown Toronto,University of Toronto


In [16]:
#Merge Toronto and DF 
New_Toronto = pd.merge(toronto, df)
New_Toronto

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
1,M4H,East York,Thorncliffe Park,43.705369,-79.349372
2,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
3,M9L,North York,Humber Summit,43.756303,-79.565963
4,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
5,M9N,York,Weston,43.706876,-79.518188
6,M3J,North York,"Northwood Park, York University",43.767980,-79.487262
7,M2H,North York,Hillcrest Village,43.803762,-79.363452
8,M2J,North York,Henry Farm,43.778517,-79.346556
9,M5S,Downtown Toronto,University of Toronto,43.662696,-79.400049


In [17]:
New_Toronto.shape

(77, 5)

In [18]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(New_Toronto['Latitude'], New_Toronto['Longitude'], New_Toronto['Borough'], New_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [19]:
#Work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data
new_toronto_data = New_Toronto[New_Toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)

In [20]:
new_toronto_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
1,M5S,Downtown Toronto,University of Toronto,43.662696,-79.400049
2,M5J,Downtown Toronto,"Toronto Islands, Union Station",43.640816,-79.381752
3,M5K,Downtown Toronto,"Design Exchange, Toronto Dominion Centre",43.647177,-79.381576
4,M5T,Downtown Toronto,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049
5,M4X,Downtown Toronto,"Cabbagetown, St. James Town",43.667967,-79.367675
6,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280
7,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529
8,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
9,M5L,Downtown Toronto,Commerce Court,43.648198,-79.379817


In [21]:
address = 'Downtown Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.655115, -79.380219.


In [22]:
# create map of Downtown Toronto using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(new_toronto_data['Latitude'], new_toronto_data['Longitude'], new_toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

Now lets add the FourSquare API and being to make come inferences

In [23]:
CLIENT_ID = 'X50DKYIGFHOJH1CKDROCC4B4HVM5T5XBJ0WUSDSJMX4MTI1Z' # your Foursquare ID
CLIENT_SECRET = 'RU4P12BLBQNMOHORRKTOPRR303C3LHFVF0U5OLBSNYTMCGTK' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: X50DKYIGFHOJH1CKDROCC4B4HVM5T5XBJ0WUSDSJMX4MTI1Z
CLIENT_SECRET:RU4P12BLBQNMOHORRKTOPRR303C3LHFVF0U5OLBSNYTMCGTK


In [24]:
#Let's Find that Italian Food!
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [25]:
#Define URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=X50DKYIGFHOJH1CKDROCC4B4HVM5T5XBJ0WUSDSJMX4MTI1Z&client_secret=RU4P12BLBQNMOHORRKTOPRR303C3LHFVF0U5OLBSNYTMCGTK&ll=43.655115,-79.380219&v=20180604&query=Italian&radius=500&limit=30'

In [26]:
#Send the GET request and examine results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cd06399dd579724108b5aa5'},
 'response': {'venues': [{'id': '52f6816f11d24a43115dc834',
    'name': 'Scaddabush Italian Kitchen & Bar',
    'location': {'address': '382 Yonge St. Unit #7',
     'crossStreet': 'Gerrard',
     'lat': 43.658920292028725,
     'lng': -79.38289105381784,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.658920292028725,
       'lng': -79.38289105381784}],
     'distance': 475,
     'postalCode': 'M5B 1S8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['382 Yonge St. Unit #7 (Gerrard)',
      'Toronto ON M5B 1S8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d110941735',
      'name': 'Italian Restaurant',
      'pluralName': 'Italian Restaurants',
      'shortName': 'Italian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15

In [27]:
#Get relevant JSON data and transform to Pandas
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,52f6816f11d24a43115dc834,382 Yonge St. Unit #7,CA,Toronto,Canada,Gerrard,475,"[382 Yonge St. Unit #7 (Gerrard), Toronto ON M...","[{'label': 'display', 'lat': 43.65892029202872...",43.658920,-79.382891,M5B 1S8,ON,Scaddabush Italian Kitchen & Bar,v-1557160857
1,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,573df789498e03dd8e54b166,595 Bay St,CA,Toronto,Canada,Dundas St,266,"[595 Bay St (Dundas St), Toronto ON M5G 2C2, C...","[{'label': 'display', 'lat': 43.65616, 'lng': ...",43.656160,-79.383190,M5G 2C2,ON,Mustachio Italian Eatery,v-1557160857
2,"[{'id': '4bf58dd8d48988d1cb941735', 'name': 'F...",False,4c787c9181bca0936180fa14,CNE Midway,CA,Toronto,Canada,NaN,338,"[CNE Midway, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.652144, 'lng':...",43.652144,-79.381118,NaN,ON,Classic italian style pizza food truck,v-1557160857
3,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",False,51bf3866498e55ee55df8db0,NaN,CA,Toronto,Canada,NaN,618,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65499143746528...",43.654991,-79.387897,NaN,ON,The Fresh Italian,v-1557160857
4,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",False,4b4a2d09f964a520687d26e3,220 Yonge St.,CA,Toronto,Canada,in Toronto Eaton Centre,79,"[220 Yonge St. (in Toronto Eaton Centre), Toro...","[{'label': 'display', 'lat': 43.655518, 'lng':...",43.655518,-79.381030,M5B 2H1,ON,Sbarro,v-1557160857


In [28]:
#This looks messy, lets filter the information and define what we want in this Dataframe
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Scaddabush Italian Kitchen & Bar,Italian Restaurant,382 Yonge St. Unit #7,CA,Toronto,Canada,Gerrard,475,"[382 Yonge St. Unit #7 (Gerrard), Toronto ON M...","[{'label': 'display', 'lat': 43.65892029202872...",43.658920,-79.382891,M5B 1S8,ON,52f6816f11d24a43115dc834
1,Mustachio Italian Eatery,Italian Restaurant,595 Bay St,CA,Toronto,Canada,Dundas St,266,"[595 Bay St (Dundas St), Toronto ON M5G 2C2, C...","[{'label': 'display', 'lat': 43.65616, 'lng': ...",43.656160,-79.383190,M5G 2C2,ON,573df789498e03dd8e54b166
2,Classic italian style pizza food truck,Food Truck,CNE Midway,CA,Toronto,Canada,NaN,338,"[CNE Midway, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.652144, 'lng':...",43.652144,-79.381118,NaN,ON,4c787c9181bca0936180fa14
3,The Fresh Italian,Italian Restaurant,NaN,CA,Toronto,Canada,NaN,618,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65499143746528...",43.654991,-79.387897,NaN,ON,51bf3866498e55ee55df8db0
4,Sbarro,Pizza Place,220 Yonge St.,CA,Toronto,Canada,in Toronto Eaton Centre,79,"[220 Yonge St. (in Toronto Eaton Centre), Toro...","[{'label': 'display', 'lat': 43.655518, 'lng':...",43.655518,-79.381030,M5B 2H1,ON,4b4a2d09f964a520687d26e3
5,Consolato Italiano,Government Building,Beverly,CA,Toronto,Canada,Dundas,370,"[Beverly (Dundas), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.6540714381019,...",43.654071,-79.375850,NaN,ON,516c1ef3e4b0b595c55461aa


In [29]:
#This looks much better. Lets visualize the Italian restaurants and map them.
dataframe_filtered.name

0          Scaddabush Italian Kitchen & Bar
1                  Mustachio Italian Eatery
2    Classic italian style pizza food truck
3                         The Fresh Italian
4                                    Sbarro
5                        Consolato Italiano
Name: name, dtype: object

In [31]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Downtown Toronto

# add a red circle marker to represent center of Downtown
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Downtown Toronto',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map